In [1]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding="utf-8")
test_df = pd.read_csv("test.csv", encoding="utf-8")
test_df


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [2]:
# 分特徵 1.數字:age, sibsp, parch, fare　2.類別:pclass,name(mid),sex,embarked 3.remove: ticket, cabin 3.drop: passid, survive, cabin, ticket
# step 1 - 補缺失值　1.數值:中位數(最可能)　2.類別:最多的值
train_df.isna().sum() #check

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [3]:
# step 1 - fillna with med in all values
med = train_df.median() #pclass is not value type
train_df = train_df.fillna(med)
test_df = test_df.fillna(med)

In [4]:
train_df.isna().sum() #check as Embarked left

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
# step 1
# fillna with Enbarked with most(s)
# most = train_df['Embarked'].value_counts() # check counts 

most = train_df["Embarked"].value_counts().idxmax()
train_df["Embarked"] = train_df["Embarked"].fillna(most)
test_df["Embarked"] = test_df["Embarked"].fillna(most)

In [6]:
test_df.isna().sum() #check as Caabin left

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64

In [7]:
# steps2 特徵處理 > 變數個"是非題" 
## step2-1 prepare for def below
n = "Braund, Mr. Owen Harris"
n = n.split(",")[-1].split(".")[0]
n.strip() #remove space/tab around str

'Mr'

In [8]:
#step2-2 to get "Mr/Miss...""
def nameflow(n):
    n = n.split(",")[-1].split(".")[0]
    return n.strip()
nametitlecount = train_df["Name"].apply(nameflow).value_counts()

reserved = nametitlecount[nametitlecount > 30].index # simple way: set list with mr. miss mannually
reserved

Index(['Mr', 'Miss', 'Mrs', 'Master'], dtype='object')

In [9]:
#step2-3 adj. name title to df
def nameflow2(n):
    n = n.split(',')[-1].split('.')[0]
    n = n.strip()
    if n in reserved:
        return n
    else:
        return None
train_df["Name"] = train_df["Name"].apply(nameflow2)
test_df["Name"] = test_df["Name"].apply(nameflow2)
test_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,Mr,male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,Mrs,female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,Mr,male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,Mr,male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,Mrs,female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,Mr,male,28.0,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,None,female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,Mr,male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,Mr,male,28.0,0,0,359309,8.0500,NaN,S


In [10]:
#step2-4 "one-hot encoding"(turing str to int) for raw_fit_data
x_train_nodrop = pd.get_dummies(train_df, 
                                columns=["Name", "Sex", "Embarked"])
x_test_nodrop = pd.get_dummies(test_df, 
                                columns=["Name", "Sex", "Embarked"])
x_train_nodrop

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Ticket,Fare,Cabin,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,22.0,1,0,A/5 21171,7.2500,NaN,0,0,1,0,0,1,0,0,1
1,2,1,1,38.0,1,0,PC 17599,71.2833,C85,0,0,0,1,1,0,1,0,0
2,3,1,3,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,1,0,0,1,0,0,0,1
3,4,1,1,35.0,1,0,113803,53.1000,C123,0,0,0,1,1,0,0,0,1
4,5,0,3,35.0,0,0,373450,8.0500,NaN,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,27.0,0,0,211536,13.0000,NaN,0,0,0,0,0,1,0,0,1
887,888,1,1,19.0,0,0,112053,30.0000,B42,0,1,0,0,1,0,0,0,1
888,889,0,3,28.0,1,2,W./C. 6607,23.4500,NaN,0,1,0,0,1,0,0,0,1
889,890,1,1,26.0,0,0,111369,30.0000,C148,0,0,1,0,0,1,1,0,0


In [11]:
#step2-5 adj. "raw_fit_data" to "fit_data" by ".drop"
x_train = x_train_nodrop.drop(["PassengerId", "Survived", "Cabin", "Ticket"], 
                               axis=1)
y_train = x_train_nodrop["Survived"]
x_test = x_test_nodrop.drop(["PassengerId", "Cabin", "Ticket"], 
                               axis=1)
testid = x_test_nodrop["PassengerId"]
x_test

,Pclass,Age,SibSp,Parch,Fare,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,34.5,0,0,7.8292,0,0,1,0,0,1,0,1,0
1,3,47.0,1,0,7.0000,0,0,0,1,1,0,0,0,1
2,2,62.0,0,0,9.6875,0,0,1,0,0,1,0,1,0
3,3,27.0,0,0,8.6625,0,0,1,0,0,1,0,0,1
4,3,22.0,1,1,12.2875,0,0,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,3,28.0,0,0,8.0500,0,0,1,0,0,1,0,0,1
414,1,39.0,0,0,108.9000,0,0,0,0,1,0,1,0,0
415,3,38.5,0,0,7.2500,0,0,1,0,0,1,0,0,1
416,3,28.0,0,0,8.0500,0,0,1,0,0,1,0,0,1


In [12]:
# STEP3-1 - looking for "best param" by GridSearchCV (repete CV)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
params = {
    "n_estimators":range(25, 40),
    "max_depth":range(6, 11)
}
clf = RandomForestClassifier()
cv = GridSearchCV(clf, params, cv=10, n_jobs=4)
cv.fit(x_train, y_train)
print(cv.best_params_)
print(cv.best_score_)

{'max_depth': 9, 'n_estimators': 30}
0.8462796504369537


In [13]:
# STEP3-2 - set up clf(分類器) with best param, + train 4 times

import numpy as np
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(n_estimators=26, max_depth=8)# n_estimators: n個分類器
score = cross_val_score(clf, x_train, y_train, cv=10, n_jobs=4)# cv:n次交叉驗證, n_jobs:number of CPUs to do the computation
print(score)
print(np.average(score))

[0.8        0.8988764  0.74157303 0.85393258 0.87640449 0.84269663
 0.82022472 0.78651685 0.84269663 0.83146067]
0.8294382022471911


In [14]:
#Step4 - train & predict
clf = RandomForestClassifier(n_estimators=26, max_depth=8)
clf.fit(x_train, y_train)
pre = clf.predict(x_test)
result_df = pd.DataFrame({
    "PassengerId":testid,
    "Survived":pre
})
result_df.to_csv("rf.csv", index=False, encoding="utf-8")
result_df

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [15]:
pd.DataFrame({
    "column":x_train.columns,
    "importance":clf.feature_importances_ #decisionTree only, while KNN NOT
})

,column,importance
0,Pclass,0.094147
1,Age,0.122965
2,SibSp,0.061017
3,Parch,0.037007
4,Fare,0.167190
5,Name_Master,0.017175
6,Name_Miss,0.020954
7,Name_Mr,0.143604
8,Name_Mrs,0.023297
9,Sex_female,0.194888


In [16]:
# KNN Algorithm
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train_scale = scaler.fit_transform(x_train)
x_test_scale = scaler.transform(x_test)

In [17]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier()
params = {
    'n_neighbors':range(5,100)
}
GridSearchCV(clf, params, cv=10, n_jobs=4)
cv.fit(x_train_scale, y_train)
print(cv.best_params_)
print(cv.best_score_)

{'max_depth': 8, 'n_estimators': 35}
0.8395630461922596


In [18]:
pre = cv.best_estimator_.predict(x_test_scale)
result_df = pd.DataFrame({
    "PassengerId":testid,
    "Survived":pre
})
result_df.to_csv("knn.csv", index=False, encoding="utf-8")
result_df

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
